<a href="https://colab.research.google.com/github/Felipehonorato1/NaturalLanguageProcessing/blob/main/GPT2Finetune_DEFAULT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 
!pip install transformers
#!pip install datasets
#!unzip tripadvisor_hotel_reviews.zip

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!nvidia-smi

Wed Mar 24 15:13:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [54]:
from transformers import GPT2Tokenizer, AutoModelWithLMHead, set_seed, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, EarlyStoppingCallback
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
set_seed(301)

In [55]:
def build_text_file(sentences, filename):
  with open(filename, 'w') as handle:
    handle.write("<|endoftext|>".join(sentences))
  

In [56]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv', sep =',')
sentences = df['Review'].values

In [57]:
train_sents, val_sents = train_test_split(sentences, test_size = 0.2, random_state = 301)

In [58]:
build_text_file(train_sents, filename = 'train.txt')
build_text_file(val_sents, filename = 'val.txt')

In [59]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [60]:
def load_dataset(train_path, test_path, tokenizer):
  train_dataset = TextDataset(tokenizer = tokenizer, file_path= train_path, block_size = 128)
  test_dataset = TextDataset(tokenizer = tokenizer, file_path= test_path, block_size = 128)
  collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

  return train_dataset, test_dataset, collator

In [61]:
train_dataset, test_dataset, collator = load_dataset(train_path= 'train.txt', test_path='val.txt', tokenizer= tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,




---


# TRAINING STEP

In [62]:
training_args = TrainingArguments(output_dir = '/content/drive/MyDrive/GPT2Out/gpt2def',num_train_epochs = 50, 
                                  per_device_train_batch_size = 32, per_device_eval_batch_size = 64, 
                                  overwrite_output_dir = True,
                                  warmup_steps = 500, evaluation_strategy = 'epoch',
                                  save_steps=-1, load_best_model_at_end=True)

In [63]:
%%capture
model = AutoModelWithLMHead.from_pretrained('gpt2')

In [64]:
early_stop = EarlyStoppingCallback()

In [65]:
trainer = Trainer(train_dataset = train_dataset, eval_dataset= test_dataset, data_collator = collator ,model = model, args= training_args, callbacks=[early_stop])

In [66]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,6.608467,3.196700,75.391000
2,No log,6.128700,3.084300,78.138000
3,No log,5.748896,3.052800,78.944000
4,No log,5.520887,3.121400,77.210000
5,No log,5.392160,3.085200,78.116000
6,No log,5.311390,3.048400,79.058000
7,No log,5.254274,3.100000,77.743000
8,No log,5.210011,3.062800,78.686000
9,No log,5.176951,3.088800,78.025000
10,No log,5.150895,3.059000,78.783000


TrainOutput(global_step=612, training_loss=5.307463801764195, metrics={'train_runtime': 969.5035, 'train_samples_per_second': 1.857, 'total_flos': 1870013739368448.0, 'epoch': 17.0, 'init_mem_cpu_alloc_delta': 49737, 'init_mem_gpu_alloc_delta': 515853312, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1144860, 'train_mem_gpu_alloc_delta': 2035973632, 'train_mem_cpu_peaked_delta': 154637960, 'train_mem_gpu_peaked_delta': 8068160512})

In [67]:
trainer.save_model()



---


# GENERATING 

In [68]:
from transformers import pipeline

reviewer = pipeline('text-generation', model = '/content/drive/MyDrive/GPT2Out/gpt2def/', tokenizer = 'distilgpt2', config={'max_length':800})

In [73]:
reviewer('i really')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'i really wish stayed seattle, staff member helpful make time nice, bed clean comfortable rooms not clean, restaurant little funky good deal right door french restaurant.overall liked stay,  '}]